[![Roboflow Notebooks](https://media.roboflow.com/notebooks/template/bannertest2-2.png?ik-sdk-version=javascript-1.4.3&updatedAt=1672932710194)](https://github.com/roboflow/notebooks)

# How to Train YOLO11 Object Detection on a Custom Dataset

---

[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/ultralytics/ultralytics)

YOLO11 builds on the advancements introduced in YOLOv9 and YOLOv10 earlier this year, incorporating improved architectural designs, enhanced feature extraction techniques, and optimized training methods.

YOLO11m achieves a higher mean mAP score on the COCO dataset while using 22% fewer parameters than YOLOv8m, making it computationally lighter without sacrificing performance.

YOLOv11 is available in 5 different sizes, ranging from `2.6M` to `56.9M` parameters, and capable of achieving from `39.5` to `54.7` mAP on the COCO dataset.

## This is 5 Objects to Detect
 1. Books
 2. Plastic Glass
 3. Highlight Pen
 4. Scissors
 5. Glass Water

In [ ]:
from IPython.display import Image, display
import requests
from io import BytesIO

url = "https://scontent.fbkk29-8.fna.fbcdn.net/v/t1.15752-9/483796462_1479161786391663_5375903957201191614_n.jpg?_nc_cat=105&ccb=1-7&_nc_sid=9f807c&_nc_ohc=c3TxDCX8j5cQ7kNvgFpaPci&_nc_oc=AdkgVc1ByyREyMaK24qNvBTiIon_5uDuUEqk6kKHCqzWSpqY0afinyRh0JYmI82q0FXxeQYo_JbU4WbIIsatbOD6&_nc_zt=23&_nc_ht=scontent.fbkk29-8.fna&oh=03_Q7cD1wGi5msjRP_10tCD4Ej0n4QKCTFoNZh3e_otmh-wZlsZog&oe=68039603"
response = requests.get(url)
img = Image(data=response.content)
display(img)

## About dataset

We splited data into 70%train 20%valid 10%test
เพิ่ม Outputs per training example (3 เท่า)

In [ ]:
from IPython.display import Image, display
import requests
from io import BytesIO

url = "https://scontent.fbkk29-8.fna.fbcdn.net/v/t1.15752-9/483848195_507572995748975_7361795539398812679_n.png?_nc_cat=105&ccb=1-7&_nc_sid=9f807c&_nc_ohc=FK2vl9DJ3rwQ7kNvgHesG9Z&_nc_oc=Adn5md_x5PRoevdn-q5Ux4R3bxjp8MukQUCC_9oCFiDSe5P406oGqymI-HaL4NDwGkaFPqwJM-kdqixY7IOeaytd&_nc_zt=23&_nc_ht=scontent.fbkk29-8.fna&oh=03_Q7cD1wHrOPqWbOFDUOlXqt_sBaXLzkux-oBc5rMaYFHXlIkbbg&oe=6803A9B8"
response = requests.get(url)
img = Image(data=response.content)
display(img)

## Setup

### Configure API keys

To fine-tune YOLO11, you need to provide your Roboflow API key. Follow these steps:

- Go to your [`Roboflow Settings`](https://app.roboflow.com/settings/api) page. Click `Copy`. This will place your private key in the clipboard.
- In Colab, go to the left pane and click on `Secrets` (🔑). Store Roboflow API Key under the name `ROBOFLOW_API_KEY`.

## Metal Performance Shaders

Since Macbook don't support NVIDIA Graphfic I need to install MPS support
lets PyTorch use the Apple Silicon GPU

In [ ]:
%pip install torch torchvision

# Verify MPS avalibility

In [ ]:
import torch
print(torch.backends.mps.is_available())  # Should return True

## Install Ultralytics (YOLO)

In [ ]:
%pip install ultralytics

## Install ROBOFLOW

In [ ]:
%pip install roboflow

**NOTE:** To make it easier for us to manage datasets, images and models we create a `HOME` constant.


In [ ]:
import os 
HOME = os.getcwd()
print(HOME)

In [ ]:
%pip install python-dotenv

## Fine-tune YOLO11 on custom dataset

NOTE: When training YOLOv11, make sure your data is located in datasets. If you'd like to change the default location of the data you want to use for fine-tuning, you can do so through Ultralytics' settings.json. In this tutorial, we will use one of the datasets available on Roboflow Universe. When downloading, make sure to select the yolov11 export format.

In [ ]:
import os
from dotenv import load_dotenv
from roboflow import Roboflow

# ตรวจสอบ working directory
print("Current working directory:", os.getcwd())
# โหลดตัวแปรจากไฟล์ .env
load_dotenv()

# ดึง API key จากตัวแปรสภาพแวดล้อม
api_key = os.getenv("ROBOFLOW_API_KEY")

# ใช้ API key กับ Roboflow
rf = Roboflow(api_key=api_key)
project = rf.project("stuff-in-my-room")
dataset = project.version(3).download("yolov11")

Current working directory: /Users/bam/Documents/Object Detection 🎐
loading Roboflow workspace...


RoboflowError: {
    "error": {
        "message": "Unsupported get request. Workspace with ID \"DailyObj\" does not exist or cannot be loaded due to missing permissions.",
        "status": 404,
        "type": "GraphMethodException",
        "hint": "You can see your available workspaces by issuing a GET request to /workspaces"
    }
}

## Custom Training

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolo11s.pt data={dataset.location}/data.yaml epochs=50 imgsz=640 batchs=